In [ ]:
# Google Driveをマウントと作業用ディレクトリ作成
from google.colab import drive 
drive.mount('/content/drive')
!mkdir -p '/content/drive/My Drive/work/'
# ファインチューニングした結果を保存するためのディレクトリ作成
!mkdir -p '/content/drive/MyDrive/work/output/'
%cd '/content/drive/My Drive/work/'

In [ ]:
# transformersをインストール
!pip install git+https://github.com/huggingface/transformers
# rinna/japanese-gpt2-mediumのトークナイザーはsentencepieceなのでsentencepieceもインストール
!pip install folium==0.2.1
!pip install sentencepiece
!pip install datasets

In [ ]:
# ファインチューニングにrun_clm.py利用するので、transformaersをダウンロード
!git clone https://github.com/huggingface/transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium")
tokenizer.do_lower_case = True
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt2-medium")

In [ ]:
!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=/content/drive/MyDrive/work/train.txt \
    --validation_file=/content/drive/MyDrive/work/train.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=10 \
    --save_steps=10000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=/content/drive/MyDrive/work/output/ \
    --use_fast_tokenizer=False

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium")
tokenizer.do_lower_case = True
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/work/output")
model.to(device)
model.eval()

In [ ]:
input_text = "ある日の暮方の事である。"
num_gen = 1
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
out = model.generate(input_ids, do_sample=True, min_length=100, max_length=400, pad_token_id=tokenizer.eos_token_id)
sent = tokenizer.batch_decode(out)[0]
sent = sent.replace('</s>', '')
sent = sent.replace('<unk>', '')
sent = sent.replace('。', '。\n')
print(sent.strip())

ある日の暮方の事である。
 二人は二階の座敷へ上つて行くと、襖側、正面の両の襖にもたれて、夜な夜な談笑に花を咲かせていた。
が、やがてそのうちに、その座敷の襖が一つ、やがて破れてしまつた。
それは床の間に掛けられている紗の一枚で、下の名前は何と云うものだったか知らなかつた。
二人はの間に、額縁とばかり重なり合つた懐中時計を見ては、夜長の電燈に戯れてゐた。
